<a href="https://colab.research.google.com/github/futurekasuga/sample/blob/master/line_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask==0.12.2 line-bot-sdk==1.5.0

     |████████████████████████████████| 92kB 3.6MB/s 
  Found existing installation: Flask 1.0.2
    Uninstalling Flask-1.0.2:
      Successfully uninstalled Flask-1.0.2


In [0]:
from linebot import LineBotApi
from linebot.models import TextSendMessage, TemplateSendMessage
from linebot.models import StickerSendMessage
from linebot.exceptions import LineBotApiError


linebot_api = LineBotApi('MNnBL1ZLDKHmrADbrwaShnjnkNPZ4lztIvOC4trOScpt3e209XooDPFuSi/596gIuHU9g4XgKyRYTGtsDm4GU/JdVcsLrViC8lqy3UoTu/edoyJ960fRKClkbx7nsxjtj7KifXC4M3iSfRCMzIeO6AdB04t89/1O/w1cDnyilFU=')
# parser = WebhookParser('a645f84aeb7fa4784dc9455ce0bd00f3')
#今日は夕方からテレビをみて盛り上がっていた,今日おばあちゃん家に来客があった,おばあちゃんが夜遅くまで起きている

linebot_api.push_message('U111a4314b8499710f1c9661600d4770e' , TextSendMessage(text="ここはメッセージ"))
linebot_api.push_message('U111a4314b8499710f1c9661600d4770e',StickerSendMessage(package_id='1',sticker_id='161'))


In [0]:
profile = linebot_api.get_profile('U111a4314b8499710f1c9661600d4770e')
print(profile.display_name) #-> 表示名
print(profile.user_id) #-> ユーザーID
print(profile.status_message) #-> ステータスメッセージ

友哉
U111a4314b8499710f1c9661600d4770e
いちごパスタ


In [0]:
from django.http import HttpResponseForbidden, HttpResponse

from linebot.exceptions import InvalidSignatureError
from linebot.models import (
    MessageEvent, TextMessage, FollowEvent, UnfollowEvent,
    TextSendMessage, ImageMessage, AudioMessage
)

from linebot import LineBotApi, WebhookHandler


# 各クライアントライブラリのインスタンス作成
line_bot_api = LineBotApi(channel_access_token='MNnBL1ZLDKHmrADbrwaShnjnkNPZ4lztIvOC4trOScpt3e209XooDPFuSi/596gIuHU9g4XgKyRYTGtsDm4GU/JdVcsLrViC8lqy3UoTu/edoyJ960fRKClkbx7nsxjtj7KifXC4M3iSfRCMzIeO6AdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler(channel_secret="dfd01345497b9893e9866c4686dc1582")


def callback(request):
    # signatureの取得
    signature = request.META['HTTP_X_LINE_SIGNATURE']
    body = request.body.decode('utf-8')
    try:
        # 署名の検証を行い、成功した場合にhandleされたメソッドを呼び出す
        handler.handle(body, signature)
    except InvalidSignatureError:
        return HttpResponseForbidden()
    return HttpResponse('OK')



# フォローイベントの場合の処理
@handler.add(FollowEvent)
def handle_follow(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='初めまして')
    )


# メッセージイベントの場合の処理
@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    # メッセージでもテキストの場合はオウム返しする
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text)
    )

@handler.add(MessageEvent, message=(ImageMessage, AudioMessage))
def handle_image_audio_message(event):
    # 画像と音源の場合は保存する
    content = line_bot_api.get_message_content(event.message.id)
    with open('file', 'w') as f:
        for c in content.iter_content():
            f.write(c)

In [0]:
from django.urls import path

urlpatterns = [
    path('callback', callback)
]